In [1]:
import requests
import bs4
from bs4 import BeautifulSoup
import pandas as pd

In [2]:
df = pd.read_csv('../kings_county_wines.csv',index_col=0)

In [3]:
wines = list(df['name'])

In [4]:
wines[:5]

['Sangria "Limonada" NV Palacio de Canedo',
 'Rosso NV Paolo Palumbo',
 'Anjou Blanc 2018 Domaine de Clayou',
 'Pinot Noir "Golden" 2017 Folktale*',
 'Shiraz HP Hydraulic Press 2016 David Franz']

In [5]:
test_wine = df['name'][1]

In [6]:
test_wine

'Rosso NV Paolo Palumbo'

In [115]:
def getWine_info(wine_name):
    search_terms = [x.replace('"','')+'+' for x in wine_name.split(' ')]

    wine_searcher_findURL = 'https://www.wine-searcher.com/find/'

    for term in search_terms:
        wine_searcher_findURL+=term
    wine_searcher_findURL=wine_searcher_findURL.strip('+')
    #print(wine_searcher_findURL)

    headers = {
        'User-Agent': """Mozilla/5.0 (Macintosh; Intel Mac OS X 10_14_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/79.0.3945.130 Safari/537.36""",
        'Connection': 'keep-alive',
        'Accept-Language': 'en-US,en;q=0.9',
        "Accept-Encoding": "gzip, deflate, br",
        "Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9",
        'Upgrade-Insecure-Requests': '1',
        'cookie': 'cookie_enabled=true',
        #'referer': f'{wine_searcher_findURL}',
        'sec-fetch-mode': 'navigate',
        'sec-fetch-site': 'same-origin',
        'cache-control': 'max-age=0',
        "sec-fetch-user": "?1"


    }

    page = requests.get(wine_searcher_findURL,headers=headers)
    #print(page)
    bs = BeautifulSoup(page.content,'lxml')
    try:
        wine_info_panel = bs.find('div',attrs={"class":'wine-info-panel'})

        wine_info = []
        for div in wine_info_panel.find_all('div',attrs={'class':'dtc'}):
            wine_info.append([x.getText() for x in div.find_all('span',attrs={'class':"dtlbl sidepanel-text"}) if type(x)==bs4.element.Tag])

        wine_info = [item.split('\n') for sublist in wine_info for item in sublist]

        wine_info = [item.strip() for sublist in wine_info for item in sublist if item not in  ['',' ']]

        wine_dict = {'name':wine_name}
        wine_dict_keys = ['Region/Appellation','Grape/Blend','Food Suggestion','Wine Style','Notes']

        for key in wine_dict_keys:
            try:
                key_index = wine_info.index(key)
                wine_dict[key]=wine_info[key_index+1]
            except:
                wine_dict[key]='null'
    except:
        print(page,wine_searcher_findURL)
        wine_dict=None
    
    return wine_dict

In [101]:
test_wine

'Rosso NV Paolo Palumbo'

In [102]:
getWine_info(test_wine)

{'name': 'Rosso NV Paolo Palumbo',
 'Region/Appellation': 'Penisola Sorrentina Lettere',
 'Grape/Blend': 'Aglianico - Piedirosso',
 'Food Suggestion': 'Beef and Venison',
 'Wine Style': 'Red - Bold and Structured',
 'Notes': 'Indicative blend: Piedirosso, Aglianico and Sciascinoso'}

In [66]:
wines = [wine.strip('*') for wine in wines]

In [73]:
def chunk(n,list_to_chunk):
    """
    takes in n, and a list to chunk. returns a list of lists with n length. The last chunk size may or may not 
    be equal to n. 
    """
    return [ list_to_chunk[i:i+n] for i in range(0,len(list_to_chunk),n) ]

In [123]:
wine_dicts = [x for x in wine_dicts if x != None]

In [124]:
import time

In [125]:
already_added = {x['name'] for x in wine_dicts}
wines_toAdd = list(set(wines)-already_added)
count = 1
wine_chunks = chunk(7,wines_toAdd)
for wine_chunk in wine_chunks:
    print(f"starting chunk {count} out of {len(wine_chunks)}")
    for wine in wine_chunk:
        wine_dicts.append(getWine_info(wine))
        count+=1
        time.sleep(5)
    
    

starting chunk 1 out of 51
<Response [200]> https://www.wine-searcher.com/find/Catarratto+L'Isola+del+Satiro+2018+Alcesti
<Response [200]> https://www.wine-searcher.com/find/Piccolo+Derthona+2017+Terralba
<Response [200]> https://www.wine-searcher.com/find/Bordeaux+Rouge+Tellus+Vinea+2018+Figeac
<Response [200]> https://www.wine-searcher.com/find/Bandol+Rouge+1.5L+2011+Châteaux+Pradeaux
<Response [200]> https://www.wine-searcher.com/find/Lambrusco+Ancestrale+2018+Il+Folicello
<Response [200]> https://www.wine-searcher.com/find/Champagne+Brut+Réserve+NV+Bouché+Père+et+Fils
starting chunk 8 out of 51
<Response [200]> https://www.wine-searcher.com/find/Refošk+Baracija+2017+Clai+Wines
<Response [200]> https://www.wine-searcher.com/find/Vinho+Verde+Rosé+2018+Lago
starting chunk 15 out of 51
<Response [200]> https://www.wine-searcher.com/find/Malvasia+Baracija+2017+Clai+Wines
<Response [200]> https://www.wine-searcher.com/find/Sauvignon+Blanc+Pouch+1.5L+Maivino
<Response [403]> https://ww

KeyboardInterrupt: 

In [126]:
wine_dicts

[{'name': 'Sangria "Limonada" NV Palacio de Canedo',
  'Region/Appellation': 'Castilla y Leon',
  'Grape/Blend': 'Mencia',
  'Food Suggestion': 'Lamb',
  'Wine Style': 'Red - Rich and Intense',
  'Notes': 'null'},
 {'name': 'Rosso NV Paolo Palumbo',
  'Region/Appellation': 'Penisola Sorrentina Lettere',
  'Grape/Blend': 'Aglianico - Piedirosso',
  'Food Suggestion': 'Beef and Venison',
  'Wine Style': 'Red - Bold and Structured',
  'Notes': 'Indicative blend: Piedirosso, Aglianico and Sciascinoso'},
 {'name': 'Anjou Blanc 2018 Domaine de Clayou',
  'Region/Appellation': 'Anjou',
  'Grape/Blend': 'Chenin Blanc',
  'Food Suggestion': 'null',
  'Wine Style': 'null',
  'Notes': 'null'},
 {'name': 'Pinot Noir "Golden" 2017 Folktale',
  'Region/Appellation': 'Monterey County',
  'Grape/Blend': 'Pinot Noir',
  'Food Suggestion': 'Chicken and Turkey',
  'Wine Style': 'Red - Light and Perfumed',
  'Notes': 'null'},
 {'name': 'Shiraz HP Hydraulic Press 2016 David Franz',
  'Region/Appellation': 

In [129]:
df = pd.DataFrame.from_records([x for x in wine_dicts if x != None])

In [132]:
df.to_csv('wine_searcher_data.csv')